<a href="https://colab.research.google.com/github/theanhvu1998/VuTheAnh_Assignment/blob/master/Copy_of_Week5_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [25]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
sentiment = pd.read_csv("/content/labeledTrainData.tsv", delimiter='\t', error_bad_lines=False, encoding='latin-1')
sentiment

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary BuÃ±uel made in the...


## 2. Preprocessing

In [28]:
# stop words
import nltk
nltk.download('stopwords')
# Removing stopwords
def remove_stop_words(text):
  return [item for item in text if item not in stop_words]
   # Your code here

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [29]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    # text = None # Your code here
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    
    return text

In [30]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here

porter = PorterStemmer()

def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]
   # Your code here

In [31]:
# split the dataset in train and test
# Your code here
sentiment["CleanSentimentText"] = sentiment["review"].apply(preprocessor)
sentiment
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [33]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.model_selection import train_test_split

X = sentiment['review']
y = sentiment['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

In [35]:
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7f5f130ff1e0>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7f5f130ff6a8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

In [36]:
p_predict = clf.predict_proba(X_test)
f_predict = clf.predict(X_test)
p_predict = pd.DataFrame(p_predict).reindex().rename(columns={0: "Negative",1: "Positive"}, errors="raise")
f_predict = pd.DataFrame(f_predict).rename(columns={0: "Predict"}, errors="raise")
predict = pd.concat([X_test.reset_index(), y_test.reset_index(),f_predict.reindex(),p_predict], axis=1)
predict.drop("index", axis=1)

,review,sentiment,Predict,Negative,Positive
0,Great acting on the part of Gretchen Mol. This...,1,1,0.043560,0.956440
1,While this movie isn't a classic by any stretc...,1,1,0.209767,0.790233
2,While I had wanted to se this film since the f...,1,1,0.107391,0.892609
3,"This sleek, sexy movie is a must-see. Only upo...",0,1,0.057918,0.942082
4,Went to Wal-Mart and found this film on DVD an...,1,1,0.114966,0.885034
...,...,...,...,...,...
4995,If you want to watch a real 'quality' movie ge...,0,0,0.971665,0.028335
4996,"Well...overall, this movie was pretty much wor...",0,0,0.798273,0.201727
4997,I cannot say this movie is a disappointment be...,1,1,0.338606,0.661394
4998,"In the Comic, Modesty is strong. Alexandra Sta...",0,1,0.495003,0.504997


## 4. Evaluate Model

In [37]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.8854
confusion matrix:
 [[2164  339]
 [ 234 2263]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.86      0.88      2503
           1       0.87      0.91      0.89      2497

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000



## 5. Export Model 

In [38]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open('logisticRegression.pkl', 'wb'))

In [39]:
with open('logisticRegression.pkl', 'rb') as model:
    reload_model = pickle.load(model)
preds = reload_model.predict_proba(X_test)

p_predict = clf.predict_proba(X_test)
f_predict = clf.predict(X_test)
p_predict = pd.DataFrame(p_predict).reindex().rename(columns={0: "Negative",1: "Positive"}, errors="raise")
f_predict = pd.DataFrame(f_predict).rename(columns={0: "Predict"}, errors="raise")
predict = pd.concat([X_test.reset_index(), y_test.reset_index(),f_predict.reindex(),p_predict], axis=1)
predict.drop("index", axis=1)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,review,sentiment,Predict,Negative,Positive
0,Great acting on the part of Gretchen Mol. This...,1,1,0.043560,0.956440
1,While this movie isn't a classic by any stretc...,1,1,0.209767,0.790233
2,While I had wanted to se this film since the f...,1,1,0.107391,0.892609
3,"This sleek, sexy movie is a must-see. Only upo...",0,1,0.057918,0.942082
4,Went to Wal-Mart and found this film on DVD an...,1,1,0.114966,0.885034
...,...,...,...,...,...
4995,If you want to watch a real 'quality' movie ge...,0,0,0.971665,0.028335
4996,"Well...overall, this movie was pretty much wor...",0,0,0.798273,0.201727
4997,I cannot say this movie is a disappointment be...,1,1,0.338606,0.661394
4998,"In the Comic, Modesty is strong. Alexandra Sta...",0,1,0.495003,0.504997
